# LangGraph Advanced – Part 1️⃣: Code Ingestion & Summarization

Notebook này hướng dẫn bạn xây dựng **pipeline LangGraph** để thu thập mã nguồn (.py) và tóm tắt chức năng bằng Claude.  
**Yêu cầu**  
* `pip install langgraph langchain langchain-anthropic`  
* Biến môi trường `ANTHROPIC_API_KEY`  
* Thư mục mã nguồn mẫu – chỉnh `CODE_DIR`

In [ ]:
%pip install -q langgraph langchain langchain-anthropic tiktoken

In [ ]:
             from pathlib import Path
             from langgraph.graph import StateGraph, MessageState
             from langchain_anthropic import ChatAnthropic
             from langchain.schema import HumanMessage

             CODE_DIR = Path("sample_code")  # ← chỉnh đường dẫn thư mục chứa .py
             llm = ChatAnthropic(model_name="claude-3-haiku-20240307", temperature=0.2)

             class CodeState(MessageState):
                 files: list[str] = []
                 summaries: dict[str, str] = {}

             def list_files(_: CodeState):
                 paths = list(CODE_DIR.glob("*.py"))
                 return CodeState(files=[str(p) for p in paths], summaries={})

             def summarize(state: CodeState):
                 for fp in state.files:
                     code = Path(fp).read_text()
                     prompt = f"Tóm tắt chức năng file Python sau:
```python
{code}
```"
                     summary = llm([HumanMessage(content=prompt)]).content
                     state.summaries[fp] = summary
                 return state

             g = StateGraph(CodeState)
             g.add_node("list", list_files)
             g.add_node("summarize", summarize)
             g.set_entry_point("list")
             g.add_edge("list", "summarize")
             pipeline = g.compile()
             result = pipeline.invoke(CodeState())
             for k, v in result.summaries.items():
                 print(k, "→", v[:120], "...")
